In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
from google.colab import files

# Lhr-dept

In [ ]:
url="http://lhr.nu.edu.pk/faculty/"
r=requests.get(url)
htmlcontent=r.content
# print(htmlcontent)
soup= BeautifulSoup(htmlcontent, 'html.parser')
# print(soup)
# print(soup.prettify)
# print(soup.title)
# print(soup.title.string)
f = soup.findAll('div', class_='facultyCard')
# print(f)
id_=[]
name=[]
email=[]
dept=[]
desig=[]
hec=[]
ext=[]
edu=[]
img=[]


for i in range(len(f)):
    faculty = f[i]
    try:
        id_.append(int(faculty.find('a',class_='faculty-link')['href'].split('/')[-1]))
    except:
        id_.append(-1)
    name.append(faculty.find('h5', class_='text-center').text)
    desig.append(faculty.find('p',class_='small').text.strip().split('\n')[0])
    try:
        x=faculty.find('p',class_='small').text.strip().split('\n')[2]
        hec.append(True)
    except:
        hec.append(False)

    email.append(faculty.find('p', class_='mb-0').text)
    dept.append(faculty.find('a',class_='faculty-link')['href'].split('/')[1])
    img.append('https://lhr.nu.edu.pk'+faculty.find('img', class_='card-img-top')['src'])
    xy='https://lhr.nu.edu.pk'+faculty.find('a',class_='faculty-link')['href']
    rr=requests.get(xy)
    htmlcontent2=rr.content
    soup2= BeautifulSoup(htmlcontent2, 'html.parser')
    ff=soup2.find('span', class_='small').text.split(':')[1]
    ext.append(ff)
    xz=soup2.find('div',class_='text-justify')
    try:
      edu.append(xz.find('li').text)
    except:
      edu.append(None)
df = pd.DataFrame({'ID':id_,
                  'Name':name,
                  'Designation':desig,
                   'HEC Approved Phd Supervisor':hec,
                   'Highest Education': edu,
                  'Email':email,
                   'Department': dept,
                   'Extension': ext,
                  'Image':img})

df.to_csv("lhr.csv",index=False)
files.download("lhr.csv")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Khi-dept

In [ ]:
url2='https://khi.nu.edu.pk/'
r2=requests.get(url2)
html2=r2.content
soup22=BeautifulSoup(html2,'html.parser')
f2=soup22.find('div',class_='kingster-main-menu')
f22=f2.find('ul',class_='sf-menu')
f222=f22.findAll('li',class_='menu-item-type-custom')
obj=f222[8]
newobj=obj.findAll('li','menu-item-type-post_type')
dfs=[]
# this new newobj object has a list of all 7 disciplines (departements) now
for i in range(len(newobj)):
  id_=[]
  name=[]
  email=[]
  dept=[]
  desig=[]
  hec=[]
  ext=[]
  edu=[]
  img=[]
  null=0
  newurl=newobj[i].find('a')['href']
  r=requests.get(newurl)
  html=r.content
  soup=BeautifulSoup(html,'html.parser')
  f=soup.findAll('div',class_='gdlr-core-personnel-list')
  for j in range(len(f)):
    faculty=f[j]
    x=faculty.find('h3',class_='gdlr-core-personnel-list-title')
    name.append(x.find('a').text.split(',')[0])
    x=faculty.find('div',class_='kingster-personnel-info-list kingster-type-email').text
    if('@' in x):
      email.append(x)
    else:
      email.append(None)
    ext.append(faculty.find('div',class_='kingster-personnel-info-list kingster-type-phone').text)
    desig.append(faculty.find('div',class_='gdlr-core-personnel-list-position gdlr-core-info-font gdlr-core-skin-caption').text)
    yz=faculty.find('div',class_='gdlr-core-media-image')
    try:
      img.append(yz.find('img')['src'])
    except:
      img.append(None)
    link=faculty.find('a',class_='gdlr-core-personnel-list-button gdlr-core-button')['href']
    rr=requests.get(link)
    htmll=rr.content
    soup2=BeautifulSoup(htmll,'html.parser')
    xx=soup2.find('span',class_='gdlr-core-title-item-caption gdlr-core-info-font gdlr-core-skin-caption').text.split('\n')
    if(len(xx)==3 and xx[2].lower()=='hec approved phd supervisor'):
      hec.append(True)
    else:
      hec.append(False)

    js_code=soup2.find('script', id='burst-js-extra').text
    pattern = r'"page_id":"(\d+)"'
    match = re.search(pattern, js_code)
    if match:
      page_id = match.group(1)  # Extract the captured value
      id_.append(page_id)
    else:
      id_.append(None)



    try:
      extra=0
      xxx=soup2.find('span',class_='gdlr-core-icon-list-content')
      pattern = r'\b(BS|MS|MCS|MPhil|PhD|ME|BE|PostDoc)\b'
      match = re.search(pattern, xxx.text)
      if match:
       edu.append(soup2.find('span',class_='gdlr-core-icon-list-content').text)
      else:
        xxx1=soup2.findAll('span',class_='gdlr-core-icon-list-content')
        pattern = r'\b(BS|MS|MCS|MPhil|PhD|ME|BE|PostDoc)\b'
        for span_element in xxx1:
          match = re.search(pattern, span_element.text)
          if match:
            edu.append(span_element.text)
            extra=1
            null=1
            break
        if(null!=1):
          xxx1=soup2.findAll('span',class_='gdlr-core-title-item-caption')
          pattern = r'\b(BS|MS|MCS|MPhil|PhD|ME|BE|PostDoc)\b'
          for span_element in xxx1:
            match= re.search(pattern,span_element.text)
            if match:
              edu.append(span_element.text)
              extra=1
              break
        if(extra==0):
          edu.append(None)
    except:
      edu.append(None)

    if(newurl.split("/")[3]=='faculty-php'):
      dept.append('fsc')
    elif(newurl.split("/")[3]=='department-of-cyber-security'):
      dept.append('cys')
    elif(newurl.split("/")[3]=='department-of-artificial-intelligence'):
      dept.append('ai')
    elif(newurl.split("/")[3]=='department-of-software-engineering'):
      dept.append('se')
    elif(newurl.split("/")[3]=='department-of-electrical-engineering'):
      dept.append('ee')
    elif(newurl.split("/")[3]=='department-of-management-sciences'):
      dept.append('fsm')
    elif(newurl.split("/")[3]=='department-of-sciences-humanities'):
      dept.append('ss')
    else:
      dept.append(None)
# print(len(id_),len(name),len(desig),len(hec),len(edu),len(email),len(dept),len(ext),len(img))
  df=pd.DataFrame({'ID':id_,
                  'Name':name,
                  'Designation':desig,
                   'HEC Approved Phd Supervisor':hec,
                   'Highest Education': edu,
                  'Email':email,
                   'Department': dept,
                   'Extension': ext,
                  'Image':img})
  dfs.append(df)
big_df = pd.concat(dfs, ignore_index=True)
big_df.to_csv("khi.csv",index=False)
files.download("khi.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Isb-dept

In [ ]:
# %cd sample_data
HTMLFile = open("FAST NUCES Islamabad.html", "r")
index = HTMLFile.read()
soup=BeautifulSoup(index,'html.parser')
# print(soup)
x=soup.findAll('div',class_='our-team')
id_=[]
name=[]
email=[]
dept=[]
desig=[]
hec=[]
ext=[]
edu=[]
img=[]
for xx in x:
  xxx=xx.find('h3',class_='title').text.strip()
  name.append(xxx)
  y=xx.find('span',class_='post').text.strip()
  desig.append(y)
  yy=xx.find('p').text.strip()
  email.append(yy)
  y=xx.find('img')['id']
  id_.append(y[-4:])
  yy=xx.find('img')['src']
  y=yy.replace(' ','')
  img.append('http://isb.nu.edu.pk'+y)
  y=xx.parent
  yy=y.parent
  newid=yy.get('id')
  if(newid=='facultyCS'):
    dept.append('fsc')
  elif(newid=='facultyEE'):
    dept.append('ee')
  elif(newid=='facultyMS'):
    dept.append('fsm')
  elif(newid=='facultySH'):
    dept.append('ss')
  # try:
  #   yyy=soup.find('div',class_="img-thumbnail").img["src"]
  #   print(yyy)
  # except:
  #   print(None)
  y=xx.find('a')['href']
  r=requests.get(y)
  html=r.content
  soup2=BeautifulSoup(html,'html.parser')
  x=soup2.find('i',class_='fa-phone')
  xxx=x.parent
  try:
    y=xxx.find('span').text.split(': ')[1]
    ext.append(y)
  except:
    ext.append(None)
  x=soup2.find('div',class_='profile-info')
  desired_style = {"style": "font-size:9pt;"}
  try:
    xx = x.find('label', attrs=desired_style).text
    hec.append(True)
  except:
    hec.append(False)
  edu.append(None)

  df = pd.DataFrame({'ID':id_,
                  'Name':name,
                  'Designation':desig,
                   'HEC Approved Phd Supervisor':hec,
                   'Highest Education': edu,
                  'Email':email,
                   'Department': dept,
                   'Extension': ext,
                  'Image':img})

df.to_csv("isb.csv",index=False)
files.download("isb.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>



# Step2

In [ ]:
df1 = pd.read_csv('lhr.csv')
df2 = pd.read_csv('khi.csv')
df3 = pd.read_csv('isb.csv')
combined_df = pd.concat([df1, df2, df3], axis=0)
# print(combined_df)
combined_df.to_csv('fast_faculty.csv', index=False)
files.download("fast_faculty.csv")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


# Step3

In [ ]:
faculty=pd.read_csv('fast_faculty.csv')
sample_ = faculty.sample(frac=0.5)
sample_.to_csv('sample_.csv',index=False)
files.download('sample_.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>